In [1]:
import emapy as epy
import sys;
import pandas as pd
import numpy as np
#reload(sys);
#sys.setdefaultencoding("utf8")

In [2]:
locationAreaBoundingBox = (41.3248770036,2.0520401001,41.4829908452,2.2813796997)

barris = epy.getDatabase('barris', 'geojson','../data/raw/barris.geojson', '',True, 0, 1, 'cartodb_id')  
allData = epy.getDatabaseFromOSM('restaurantes', 'amenity', False, True, locationAreaBoundingBox, 'bar')

T = [[barri, data['properties'], data['geometry']] for barri in barris[1] for data in allData if epy.coordInsidePolygon(data["geometry"][0], data["geometry"][1], epy.transformArrYXToXY(barris[1][barri]))]


In [3]:
df = pd.DataFrame({'id' : [], 'data': []})

allId = dict()

for data in T:
    key = int(float(data[0]))
    if key in allId:
        allId[key] += 1
    else:
        allId[key] = 1
     
        
for idBarri in barris[1]:
    key = int(float(idBarri))
    if key in allId:
        row = [key, allId[key] * 1.0 / len(T)]  
        df.loc[len(df), ['id', 'data']] = row  
    else:
        df.loc[len(df), ['id', 'data']] = [key,0]

map = epy.mapCreation(41.388790,2.158990)
epy.mapChoropleth(map, 
                 '../data/raw/barris.geojson',
                 'feature.properties.cartodb_id',
                 df,
                 'id',
                 'data',
                 'YlGn',
                 0.7,
                 0.3,
                 [],
                 'bars / barri')



emapy.py:950: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
  legend_name=legendName)


In [4]:
epy.mapSave(map, '../reports/maps/mapOfBarsxBarri.html')
map

In [5]:
print df['data'].max()

0.114942528736
